# Observing Imprecisions in Floating-Point Arithmetic

- Imprecisions are present in floating point arithmetic
- Machines store finite representations of data, and inexact values must be rounded
- Even "exact" values at different magnitudes cannot necessarily be used together without the introduction of error
- You can demonstrate this with Python3 out-of-the-box

In [1]:
def count_to_one(N):
    step = 1.0 / N
    return step, step*N, sum([step for _ in range(int(N))])

- The function above should return the number 1: we're adding N steps of size 1/N together

In [2]:
N = 1e3
print(count_to_one(N))

(0.001, 1.0, 1.0000000000000007)


- That didn't give us the number 1...
- It's _close_, but not exact. Is it random/probabilistic?
- Let's run it 5 more times

In [3]:
N = 1e3
for _ in range(5):
    print(count_to_one(N))

(0.001, 1.0, 1.0000000000000007)
(0.001, 1.0, 1.0000000000000007)
(0.001, 1.0, 1.0000000000000007)
(0.001, 1.0, 1.0000000000000007)
(0.001, 1.0, 1.0000000000000007)


- So we're getting the _same_ *wrong* answer each time
- Does the value we pick for N make a difference?

In [4]:
for N in [1e3, 1e6]:
    print("N = {0:.0e}".format(N))
    for _ in range(5):
        print("  {0}".format(count_to_one(N)))

N = 1e+03
  (0.001, 1.0, 1.0000000000000007)
  (0.001, 1.0, 1.0000000000000007)
  (0.001, 1.0, 1.0000000000000007)
  (0.001, 1.0, 1.0000000000000007)
  (0.001, 1.0, 1.0000000000000007)
N = 1e+06
  (1e-06, 1.0, 1.000000000007918)
  (1e-06, 1.0, 1.000000000007918)
  (1e-06, 1.0, 1.000000000007918)
  (1e-06, 1.0, 1.000000000007918)
  (1e-06, 1.0, 1.000000000007918)


- we see both under- and over-estimations that are consistent for each value of N we tried
- it so happens, decimal multiples of 10 cannot be stored in finite bits (like many, many other decimal numbers)
- the initial division in our function introduced a bit of error
- the subsequent addition of this incorrect value compounded the difference
- does this problem also exist for other datatypes/numpy?

In [5]:
import numpy as np

def count_to_one(N, dtype=float):
    N = dtype(N)
    step = dtype(1.0) / N
    return step, step*N, np.sum([step for _ in range(int(N))])

In [6]:
for N in [1e1, 1e2, 1e3, 1e6]:
    print("N = {0:.0e} ({0})".format(N))
    for dt in [int, np.int32, np.int64, float, np.float32, np.float64]:
        estimate_of_one = count_to_one(N, dt)
        print("  {0} ({1} => {2})".format(estimate_of_one[2],
                                          dt.__name__,
                                          type(estimate_of_one[2]).__name__))

N = 1e+01 (10.0)
  1.0 (int => float64)
  1.0 (int32 => float64)
  1.0 (int64 => float64)
  1.0 (float => float64)
  1.0 (float32 => float32)
  1.0 (float64 => float64)
N = 1e+02 (100.0)
  0.9999999999999999 (int => float64)
  0.9999999999999999 (int32 => float64)
  0.9999999999999999 (int64 => float64)
  0.9999999999999999 (float => float64)
  0.9999998211860657 (float32 => float32)
  0.9999999999999999 (float64 => float64)
N = 1e+03 (1000.0)
  1.0000000000000004 (int => float64)
  1.0000000000000004 (int32 => float64)
  1.0000000000000004 (int64 => float64)
  1.0000000000000004 (float => float64)
  1.0000001192092896 (float32 => float32)
  1.0000000000000004 (float64 => float64)
N = 1e+06 (1000000.0)
  0.9999999999999981 (int => float64)
  0.9999999999999981 (int32 => float64)
  0.9999999999999981 (int64 => float64)
  0.9999999999999981 (float => float64)
  1.000000238418579 (float32 => float32)
  0.9999999999999981 (float64 => float64)
